In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pymongo
from pymongo import MongoClient
import csv
import json

In [2]:
# load marathon data files
berlin_men_load = 'Male_Elite_Berlin.csv'
boston_men_load = 'Male_Elite_Boston.csv'
chicago_men_load = 'Male_Elite_Chicago.csv'
london_men_load = 'Male_Elite_London.csv'
new_york_men_load = 'Male_Elite_New_York.csv'
tokyo_men_load = 'Male_Elite_Tokyo.csv'

# read files
berlin_men = pd.read_csv(berlin_men_load, encoding='unicode_escape')
berlin_men = berlin_men.assign(Marathon='Berlin')

boston_men = pd.read_csv(boston_men_load)
boston_men = boston_men.assign(Marathon='Boston')
boston_men = boston_men[['Year','Athlete','Country/State or Province','Time','Marathon']]
boston_men = boston_men.rename(columns={'Country/State or Province':'Country'})
boston_men_sorted = boston_men.sort_values('Year', ascending=False)

chicago_men = pd.read_csv(chicago_men_load, encoding='unicode_escape')
chicago_men = chicago_men.assign(Marathon='Chicago')
chicago_men_sorted = chicago_men.sort_values('Year', ascending=False)

london_men = pd.read_csv(london_men_load, encoding='unicode_escape')
london_men = london_men.assign(Marathon='London')
london_men = london_men[['Year','Athlete', 'Nationality','Time(h:m:s)','Marathon']]
london_men = london_men.rename(columns={'Nationality':'Country','Time(h:m:s)':'Time'})
london_men_sorted = london_men.sort_values('Year', ascending=False)

new_york_men = pd.read_csv(new_york_men_load, encoding='unicode_escape')
new_york_men = new_york_men.assign(Marathon='New York')
new_york_men = new_york_men[['Year','Winner','Country','Time','Marathon']]
new_york_men = new_york_men.rename(columns={'Winner':'Athlete'})
new_york_men_sorted = new_york_men.sort_values('Year', ascending=False)

tokyo_men = pd.read_csv(tokyo_men_load, encoding='unicode_escape')
tokyo_men = tokyo_men.assign(Marathon='Tokyo')

# merge files to single data frame
mmdm = pd.concat([boston_men_sorted,new_york_men_sorted,chicago_men_sorted,berlin_men,london_men_sorted,tokyo_men])
mmdm = mmdm.reset_index()
mmdm = mmdm.drop(columns=['index'])


# export to CSV
mmdm.to_csv('major_marathon_men.csv')
mmdm.head()

,Year,Athlete,Country,Time,Marathon
0,2018,Yuki Kawauchi,Japan,02:15:58,Boston
1,2017,Geoffrey Kirui,Kenya,02:09:37,Boston
2,2016,Berhanu Hayle,Ethiopia,02:12:45,Boston
3,2015,Desisa Benti,Ethiopia,02:09:17,Boston
4,2014,Meb Keflezighi,United States,02:08:37,Boston


In [6]:
marathon_list = []
headers = ["Year", "Athlete", "Country", "Time", "Marathon"]

with open('major_marathon_men.csv') as f:
    reader = csv.reader(f)
    reader.__next__()
    for row in reader:
        marathon_list.append(dict(zip(headers,row[1:])))

In [7]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.ETL_Project
collection = db.ETL_Project

collection.insert_many(marathon_list)